In [1]:
# Import dependencies.
import pandas as pd
import numpy as np
#import hvplot.pandas
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import accuracy_score
import tensorflow as tf

In [2]:
# Urls for each city's data from S3 bucket.
url_nash = "nashville_listings.csv.gz"
url_nyc = "nyc_listings.csv.gz"
url_sd = "sd_listings.csv.gz"
url_sea = "seattle_listings.csv.gz"

In [3]:
# Nashville df
df_nash = pd.read_csv(url_nash, compression='gzip')
# NYC df
df_nyc = pd.read_csv(url_nyc, compression='gzip')
# San Diego df
df_sd = pd.read_csv(url_sd, compression='gzip')
# Seattle df
df_sea = pd.read_csv(url_sea, compression='gzip')

C:\Users\radov\AppData\Local\Temp\ipykernel_324\1333899189.py:4: DtypeWarning: Columns (68) have mixed types. Specify dtype option on import or set low_memory=False.
  df_nyc = pd.read_csv(url_nyc, compression='gzip')


In [4]:
# Create a df geared towards reviews analysis.
reviews_df = df_sd[['accommodates', 'price', 'availability_30', 'availability_60', 'availability_90', 
                   'availability_365', 'number_of_reviews', 'review_scores_rating', 'review_scores_accuracy', 
                   'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 
                   'review_scores_location', 'review_scores_value']]

reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12871 entries, 0 to 12870
Data columns (total 14 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   accommodates                 12871 non-null  int64  
 1   price                        12871 non-null  object 
 2   availability_30              12871 non-null  int64  
 3   availability_60              12871 non-null  int64  
 4   availability_90              12871 non-null  int64  
 5   availability_365             12871 non-null  int64  
 6   number_of_reviews            12871 non-null  int64  
 7   review_scores_rating         10957 non-null  float64
 8   review_scores_accuracy       10939 non-null  float64
 9   review_scores_cleanliness    10939 non-null  float64
 10  review_scores_checkin        10937 non-null  float64
 11  review_scores_communication  10939 non-null  float64
 12  review_scores_location       10937 non-null  float64
 13  review_scores_va

# Drop rows with NaN values

In [5]:
# Drop rows with null values.
reviews_df = reviews_df.dropna()

reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10937 entries, 0 to 12838
Data columns (total 14 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   accommodates                 10937 non-null  int64  
 1   price                        10937 non-null  object 
 2   availability_30              10937 non-null  int64  
 3   availability_60              10937 non-null  int64  
 4   availability_90              10937 non-null  int64  
 5   availability_365             10937 non-null  int64  
 6   number_of_reviews            10937 non-null  int64  
 7   review_scores_rating         10937 non-null  float64
 8   review_scores_accuracy       10937 non-null  float64
 9   review_scores_cleanliness    10937 non-null  float64
 10  review_scores_checkin        10937 non-null  float64
 11  review_scores_communication  10937 non-null  float64
 12  review_scores_location       10937 non-null  float64
 13  review_scores_value  

# Accommodates

In [6]:
# Check out value counts for 'accommodates' column.
reviews_df['accommodates'].value_counts()

accommodates
2     2767
4     2234
6     1625
8      905
3      743
5      640
1      534
10     411
7      317
12     217
16     198
9      142
14     100
11      56
15      30
13      18
Name: count, dtype: int64

# Price

In [5]:
# Preview values in 'price' column.
reviews_df['price'].head(5)

0      $725.00
1      $349.00
2      $225.00
3      $794.00
4    $1,250.00
Name: price, dtype: object

In [6]:
# Remove dollar signs and commas from 'price' values and change data type to float.
reviews_df['price'] = reviews_df['price'].str.replace('$', '', regex=False).str.replace(',', '').astype(float)
reviews_df['price'].head(5)

C:\Users\radov\AppData\Local\Temp\ipykernel_324\3730570469.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews_df['price'] = reviews_df['price'].str.replace('$', '', regex=False).str.replace(',', '').astype(float)


0     725.0
1     349.0
2     225.0
3     794.0
4    1250.0
Name: price, dtype: float64

In [7]:
# Create an empty list to hold our possible outliers.
outliers = []

# Calculate the IQR and quantitatively determine if there are any potential outliers in the 'price' column.
quartiles = reviews_df['price'].quantile([0.05, .25, .5, .75, 0.95])
lowerq = quartiles[0.05]
upperq = quartiles[0.95]
iqr = upperq - lowerq 
lower_bound = lowerq - (1.5 * iqr)
upper_bound = upperq + (1.5 * iqr)

In [8]:
# Determine outliers using upper and lower bounds.
for i in reviews_df['price']:
    if (i < lower_bound) | (i > upper_bound):
        outliers.append(i)

# Print out all the possible outliers.
print(f'Count: {len(outliers)}, Potential outliers: {outliers}')

Count: 92, Potential outliers: [3000.0, 3500.0, 2550.0, 2240.0, 2196.0, 3049.0, 2114.0, 2432.0, 3000.0, 3000.0, 7181.0, 3000.0, 2543.0, 5000.0, 2343.0, 2156.0, 2500.0, 2093.0, 2950.0, 2300.0, 3420.0, 3563.0, 10000.0, 2142.0, 9999.0, 2426.0, 4500.0, 2987.0, 2091.0, 2317.0, 2243.0, 2379.0, 2150.0, 100000.0, 4213.0, 2443.0, 3869.0, 2201.0, 3154.0, 2795.0, 2650.0, 2428.0, 3433.0, 2895.0, 10000.0, 2786.0, 2200.0, 3621.0, 3528.0, 2906.0, 2377.0, 3900.0, 2500.0, 2966.0, 2429.0, 2981.0, 5260.0, 2585.0, 2209.0, 3018.0, 3643.0, 2307.0, 2200.0, 8807.0, 2100.0, 2150.0, 2873.0, 2283.0, 10000.0, 2674.0, 5000.0, 2500.0, 5000.0, 3500.0, 5000.0, 3800.0, 3185.0, 3000.0, 2140.0, 10000.0, 2500.0, 7586.0, 2566.0, 3000.0, 5213.0, 3105.0, 2700.0, 5214.0, 17429.0, 2760.0, 7993.0, 5920.0]


In [9]:
# Identify outlier amounts with a 'DROP' label and then drop these rows from the df.
for amt in outliers:
    reviews_df['price'] = reviews_df['price'].replace(amt, 'DROP')

reviews_df = reviews_df.drop(reviews_df[reviews_df['price'] == 'DROP'].index)

C:\Users\radov\AppData\Local\Temp\ipykernel_324\627342180.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews_df['price'] = reviews_df['price'].replace(amt, 'DROP')


In [10]:
# Check to see that 83 rows were dropped.
len(reviews_df)

12779

In [11]:
# Change back data type of 'price' column to float.
reviews_df['price'] = reviews_df['price'].astype(float)

In [12]:
# Get an overview of the 'price' column.
reviews_df['price'].describe()

count    12779.000000
mean       277.167149
std        272.842238
min          0.000000
25%        118.000000
50%        185.000000
75%        330.000000
max       2074.000000
Name: price, dtype: float64

# Number of Reviews

In [15]:
reviews_df['number_of_reviews'].value_counts()

number_of_reviews
1      675
2      464
3      419
4      340
6      276
      ... 
597      1
706      1
606      1
420      1
313      1
Name: count, Length: 514, dtype: int64

In [16]:
# Loop through each row and bin them together accordingly.
reviews_df['num_of_revs_cat'] = reviews_df['number_of_reviews']

for i in reviews_df['num_of_revs_cat']:

    if 0 < i <= 25:
        reviews_df['num_of_revs_cat'] = reviews_df['num_of_revs_cat'].replace(i, '1-25')

    elif 25 < i <= 50:
        reviews_df['num_of_revs_cat'] = reviews_df['num_of_revs_cat'].replace(i, '26-50')

    elif 50 < i <= 100:
        reviews_df['num_of_revs_cat'] = reviews_df['num_of_revs_cat'].replace(i, '51-100')
        
    elif 100 < i <= 150:
        reviews_df['num_of_revs_cat'] = reviews_df['num_of_revs_cat'].replace(i, '101-150')
        
    elif 150 < i <= 200:
        reviews_df['num_of_revs_cat'] = reviews_df['num_of_revs_cat'].replace(i, '151-200')
        
    elif 200 < i <= 300:
        reviews_df['num_of_revs_cat'] = reviews_df['num_of_revs_cat'].replace(i, '201-300')

    else:
        reviews_df['num_of_revs_cat'] = reviews_df['num_of_revs_cat'].replace(i, '300+')

In [17]:
reviews_df['num_of_revs_cat'].value_counts()

num_of_revs_cat
1-25       5239
26-50      1669
51-100     1620
101-150     799
201-300     559
151-200     543
300+        425
Name: count, dtype: int64

# Create Revenue columns for all Availability periods

In [13]:
# Create revenue column for 30 days out.
reviews_df["revenue_30"] = (30 - reviews_df['availability_30']) * reviews_df['price']

# Create revenue column for 60 days out.
reviews_df["revenue_60"] = (60 - reviews_df['availability_60']) * reviews_df['price']

# Create revenue column for 90 days out.
reviews_df["revenue_90"] = (90 - reviews_df['availability_90']) * reviews_df['price']

# Create revenue column for 365 days out.
reviews_df["revenue_365"] = (365 - reviews_df['availability_365']) * reviews_df['price']

In [14]:
# Create revenue per accommodation column for 30 days out.
reviews_df['revenue_pp_30'] = reviews_df['revenue_30'] / reviews_df['accommodates']

# Create revenue per accommodation column for 60 days out.
reviews_df['revenue_pp_60'] = reviews_df['revenue_60'] / reviews_df['accommodates']

# Create revenue per accommodation column for 90 days out.
reviews_df['revenue_pp_90'] = reviews_df['revenue_90'] / reviews_df['accommodates']

# Create revenue per accommodation column for 365 days out.
reviews_df['revenue_pp_365'] = reviews_df['revenue_365'] / reviews_df['accommodates']

In [20]:
# Preview special outlier.
spec_outlier = reviews_df[reviews_df['revenue_pp_90'] == 126000]
spec_outlier

,accommodates,price,availability_30,availability_60,availability_90,availability_365,number_of_reviews,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,...,review_scores_value,num_of_revs_cat,revenue_30,revenue_60,revenue_90,revenue_365,revenue_pp_30,revenue_pp_60,revenue_pp_90,revenue_pp_365
1855,1,1400.0,0,0,0,0,2,5.0,5.0,5.0,...,5.0,1-25,42000.0,84000.0,126000.0,511000.0,42000.0,84000.0,126000.0,511000.0


In [15]:
# Remove this special outlier and reset the index of the df.
reviews_df = reviews_df.drop(reviews_df[reviews_df['revenue_pp_90'] == 126000].index).reset_index(drop=True)

In [16]:
reviews_df

,accommodates,price,availability_30,availability_60,availability_90,availability_365,number_of_reviews,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,...,review_scores_location,review_scores_value,revenue_30,revenue_60,revenue_90,revenue_365,revenue_pp_30,revenue_pp_60,revenue_pp_90,revenue_pp_365
0,4,725.0,28,58,88,178,15,4.93,4.93,4.64,...,5.00,4.71,1450.0,1450.0,1450.0,135575.0,362.500000,362.500000,362.500000,33893.750000
1,6,349.0,0,0,0,0,153,4.81,4.82,4.71,...,4.77,4.80,10470.0,20940.0,31410.0,127385.0,1745.000000,3490.000000,5235.000000,21230.833333
2,6,225.0,0,0,13,123,92,4.77,4.76,4.56,...,4.81,4.76,6750.0,13500.0,17325.0,54450.0,1125.000000,2250.000000,2887.500000,9075.000000
3,14,794.0,5,14,19,205,25,4.50,4.54,4.21,...,4.83,4.42,19850.0,36524.0,56374.0,127040.0,1417.857143,2608.857143,4026.714286,9074.285714
4,12,1250.0,14,31,43,261,26,4.72,4.67,4.56,...,4.79,4.58,20000.0,36250.0,58750.0,130000.0,1666.666667,3020.833333,4895.833333,10833.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12773,6,202.0,4,23,31,299,0,NaN,NaN,NaN,...,NaN,NaN,5252.0,7474.0,11918.0,13332.0,875.333333,1245.666667,1986.333333,2222.000000
12774,12,749.0,25,51,73,149,0,NaN,NaN,NaN,...,NaN,NaN,3745.0,6741.0,12733.0,161784.0,312.083333,561.750000,1061.083333,13482.000000
12775,14,356.0,16,39,59,315,0,NaN,NaN,NaN,...,NaN,NaN,4984.0,7476.0,11036.0,17800.0,356.000000,534.000000,788.285714,1271.428571
12776,4,284.0,0,23,53,327,0,NaN,NaN,NaN,...,NaN,NaN,8520.0,10508.0,10508.0,10792.0,2130.000000,2627.000000,2627.000000,2698.000000


# Place 'revenue_pp_90' column into Percentile Groups

In [18]:
# Calculate the quartiles.
quartiles = reviews_df['revenue_pp_90'].quantile([.25, .5, .75])
lowerq = quartiles[0.25]
mean = quartiles[0.5]
upperq = quartiles[0.75]
iqr = upperq - lowerq 
lower_bound = lowerq - (1.5 * iqr)
upper_bound = upperq + (1.5 * iqr)

In [19]:
# Loop through each row and bin them together accordingly.
reviews_df['rpp_90_quartile'] = reviews_df['revenue_pp_90']

for i in reviews_df['rpp_90_quartile']:
    
    if i > upper_bound:
        reviews_df['rpp_90_quartile'] = reviews_df['rpp_90_quartile'].replace(i, 3)
        
    elif upper_bound >= i > mean:
        reviews_df['rpp_90_quartile'] = reviews_df['rpp_90_quartile'].replace(i, 2)
        
    elif mean >= i > lower_bound:
        reviews_df['rpp_90_quartile'] = reviews_df['rpp_90_quartile'].replace(i, 1)
        
    else:
        reviews_df['rpp_90_quartile'] = reviews_df['rpp_90_quartile'].replace(i, 0)

In [20]:
reviews_df['rpp_90_quartile'].value_counts()

rpp_90_quartile
1.0    6392
2.0    5894
3.0     492
Name: count, dtype: int64

# Place 'revenue_pp_90' column into just two Percentile Groups

In [21]:
# Loop through each row and bin them together accordingly.
reviews_df['rpp_90_binary'] = reviews_df['revenue_pp_90']

for i in reviews_df['rpp_90_binary']:
    
    if i > mean:
        reviews_df['rpp_90_binary'] = reviews_df['rpp_90_binary'].replace(i, 1)
        
    else:
        reviews_df['rpp_90_binary'] = reviews_df['rpp_90_binary'].replace(i, 0)

In [22]:
reviews_df['rpp_90_binary'].value_counts()

rpp_90_binary
0.0    6392
1.0    6386
Name: count, dtype: int64

# Reviews Exploration

In [28]:
reviews_df.hvplot.scatter(
    x='review_scores_accuracy',
    y='revenue_pp_90',
    hover_cols='num_of_revs_cat'
)

AttributeError: 'DataFrame' object has no attribute 'hvplot'

In [ ]:
reviews_df.hvplot.scatter(
    x='review_scores_cleanliness',
    y='revenue_pp_90',
    hover_cols='num_of_revs_cat'
)

In [ ]:
reviews_df.hvplot.scatter(
    x='review_scores_checkin',
    y='revenue_pp_90',
    hover_cols='num_of_revs_cat'
)

In [ ]:
reviews_df.hvplot.scatter(
    x='review_scores_communication',
    y='revenue_pp_90',
    hover_cols='num_of_revs_cat'
)

In [ ]:
reviews_df.hvplot.scatter(
    x='review_scores_location',
    y='revenue_pp_90',
    hover_cols='num_of_revs_cat'
)

In [ ]:
reviews_df.hvplot.scatter(
    x='review_scores_value',
    y='revenue_pp_90',
    hover_cols='num_of_revs_cat'
)

# Deep Learning Preprocessing

In [29]:
# Preview data types.
reviews_df.dtypes

accommodates                     int64
price                          float64
availability_30                  int64
availability_60                  int64
availability_90                  int64
availability_365                 int64
number_of_reviews                int64
review_scores_rating           float64
review_scores_accuracy         float64
review_scores_cleanliness      float64
review_scores_checkin          float64
review_scores_communication    float64
review_scores_location         float64
review_scores_value            float64
num_of_revs_cat                 object
revenue_30                     float64
revenue_60                     float64
revenue_90                     float64
revenue_365                    float64
revenue_pp_30                  float64
revenue_pp_60                  float64
revenue_pp_90                  float64
revenue_pp_365                 float64
rpp_90_quartile                float64
rpp_90_binary                  float64
dtype: object

In [30]:
# Create categorical variable list.
cat_columns = reviews_df.dtypes[reviews_df.dtypes == "object"].index.tolist()

In [31]:
# Create a OneHotEncoder instance.
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list.
encode_df = pd.DataFrame(enc.fit_transform(reviews_df[cat_columns]))

# Add the encoded variable names to the dataframe.
encode_df.columns = enc.get_feature_names_out()
encode_df

C:\Users\radov\anaconda3\envs\tfenv\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


,num_of_revs_cat_1-25,num_of_revs_cat_101-150,num_of_revs_cat_151-200,num_of_revs_cat_201-300,num_of_revs_cat_26-50,num_of_revs_cat_300+,num_of_revs_cat_51-100
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...
10848,1.0,0.0,0.0,0.0,0.0,0.0,0.0
10849,1.0,0.0,0.0,0.0,0.0,0.0,0.0
10850,1.0,0.0,0.0,0.0,0.0,0.0,0.0
10851,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [32]:
# Merge one-hot encoded features and drop the originals.
reviews_df = reviews_df.merge(encode_df, left_index=True, right_index=True)
reviews_df = reviews_df.drop(columns=cat_columns)
reviews_df.head(1)

,accommodates,price,availability_30,availability_60,availability_90,availability_365,number_of_reviews,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,...,revenue_pp_365,rpp_90_quartile,rpp_90_binary,num_of_revs_cat_1-25,num_of_revs_cat_101-150,num_of_revs_cat_151-200,num_of_revs_cat_201-300,num_of_revs_cat_26-50,num_of_revs_cat_300+,num_of_revs_cat_51-100
0,4,725.0,28,58,88,178,15,4.93,4.93,4.64,...,33893.75,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [33]:
# Export as a CSV file.
reviews_df.to_csv('saved_csvs/reviews_ml.csv', index=False)

# Neural Network with 'rpp_90_quartile' Target

In [34]:
# Split our preprocessed data into our features and target arrays for 90 day availability.
X = reviews_df.drop(columns=['price', 'number_of_reviews', 'availability_30', 'availability_60', 
                             'availability_90', 'availability_365', 'revenue_30', 'revenue_60', 'revenue_90', 
                             'revenue_365', 'revenue_pp_30', 'revenue_pp_60', 'revenue_pp_90', 'revenue_pp_365', 
                             'rpp_90_quartile', 'rpp_90_binary'])

y = reviews_df['rpp_90_quartile']

# Split the preprocessed data into a training and testing dataset.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [35]:
# Preview features df.
X

,accommodates,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,num_of_revs_cat_1-25,num_of_revs_cat_101-150,num_of_revs_cat_151-200,num_of_revs_cat_201-300,num_of_revs_cat_26-50,num_of_revs_cat_300+,num_of_revs_cat_51-100
0,4,4.93,4.93,4.64,4.79,5.00,5.00,4.71,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,6,4.81,4.82,4.71,4.99,4.97,4.77,4.80,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,6,4.77,4.76,4.56,4.91,4.91,4.81,4.76,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,14,4.50,4.54,4.21,4.54,4.54,4.83,4.42,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,12,4.72,4.67,4.56,4.92,4.83,4.79,4.58,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10848,4,5.00,5.00,5.00,5.00,5.00,4.00,4.50,1.0,0.0,0.0,0.0,0.0,0.0,0.0
10849,6,5.00,5.00,5.00,5.00,5.00,5.00,5.00,1.0,0.0,0.0,0.0,0.0,0.0,0.0
10850,2,5.00,5.00,5.00,5.00,5.00,5.00,5.00,1.0,0.0,0.0,0.0,0.0,0.0,0.0
10851,2,5.00,5.00,5.00,5.00,5.00,5.00,5.00,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [36]:
# Create a StandardScaler instances.
scaler = StandardScaler()

# Fit the StandardScaler.
X_scaled = scaler.fit(X_train)

# Scale the data.
X_train_scaled = X_scaled.transform(X_train)
X_test_scaled = X_scaled.transform(X_test)

In [37]:
X_train_scaled[0]

array([-0.27036566,  0.68934575,  0.59202832,  0.68064293,  0.44596161,
       -2.55989793,  0.57657303,  0.83989645,  1.03692658, -0.27854096,
       -0.23267588, -0.23675502, -0.4277782 , -0.20131249, -0.41687092])

In [38]:
# Define the number of input features.
num_features = len(X.columns)

# Create the model
model = tf.keras.models.Sequential()

# First hidden layer.
model.add(tf.keras.layers.Dense(units=12, input_dim=num_features, activation='relu'))

# Second hidden layer.
model.add(tf.keras.layers.Dense(units=16, activation='relu'))

# Output layer.
model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model.
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 12)                192       
                                                                 
 dense_1 (Dense)             (None, 16)                208       
                                                                 
 dense_2 (Dense)             (None, 1)                 17        
                                                                 
Total params: 417
Trainable params: 417
Non-trainable params: 0
_________________________________________________________________


In [39]:
# Compile the model.
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [40]:
# Train the model.
fit_model = model.fit(X_train, y_train, epochs=100)

Epoch 1/100
255/255 [==============================] - 1s 1ms/step - loss: -4.7151 - accuracy: 0.3537
Epoch 2/100
255/255 [==============================] - 0s 1ms/step - loss: -189.8497 - accuracy: 0.5056
Epoch 3/100
255/255 [==============================] - 0s 1ms/step - loss: -1054.1317 - accuracy: 0.5056
Epoch 4/100
255/255 [==============================] - 0s 1ms/step - loss: -3107.5474 - accuracy: 0.5056
Epoch 5/100
255/255 [==============================] - 0s 1ms/step - loss: -6687.0396 - accuracy: 0.5056
Epoch 6/100
255/255 [==============================] - 0s 1ms/step - loss: -11972.2432 - accuracy: 0.5056
Epoch 7/100
255/255 [==============================] - 0s 1ms/step - loss: -19155.0723 - accuracy: 0.5056
Epoch 8/100
255/255 [==============================] - 0s 1ms/step - loss: -28381.6953 - accuracy: 0.5056
Epoch 9/100
255/255 [==============================] - 0s 1ms/step - loss: -39967.7891 - accuracy: 0.5056
Epoch 10/100
255/255 [==============================] -

In [41]:
# Evaluate the model using the test data.
model_loss, model_acc = model.evaluate(X_test, y_test)
print(f'Loss: {model_loss}, Accuracy: {model_acc}')

85/85 [==============================] - 0s 1ms/step - loss: -22553450.0000 - accuracy: 0.4834
Loss: -22553450.0, Accuracy: 0.4834192991256714


# Neural Network with 'rpp_90_binary' Target

In [42]:
# Split our preprocessed data into our features and target arrays for 90 day availability.
X = reviews_df.drop(columns=['price', 'number_of_reviews', 'availability_30', 'availability_60', 
                             'availability_90', 'availability_365', 'revenue_30', 'revenue_60', 'revenue_90', 
                             'revenue_365', 'revenue_pp_30', 'revenue_pp_60', 'revenue_pp_90', 'revenue_pp_365', 
                             'rpp_90_quartile', 'rpp_90_binary'])

y = reviews_df['rpp_90_binary']

# Split the preprocessed data into a training and testing dataset.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [43]:
X

,accommodates,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,num_of_revs_cat_1-25,num_of_revs_cat_101-150,num_of_revs_cat_151-200,num_of_revs_cat_201-300,num_of_revs_cat_26-50,num_of_revs_cat_300+,num_of_revs_cat_51-100
0,4,4.93,4.93,4.64,4.79,5.00,5.00,4.71,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,6,4.81,4.82,4.71,4.99,4.97,4.77,4.80,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,6,4.77,4.76,4.56,4.91,4.91,4.81,4.76,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,14,4.50,4.54,4.21,4.54,4.54,4.83,4.42,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,12,4.72,4.67,4.56,4.92,4.83,4.79,4.58,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10848,4,5.00,5.00,5.00,5.00,5.00,4.00,4.50,1.0,0.0,0.0,0.0,0.0,0.0,0.0
10849,6,5.00,5.00,5.00,5.00,5.00,5.00,5.00,1.0,0.0,0.0,0.0,0.0,0.0,0.0
10850,2,5.00,5.00,5.00,5.00,5.00,5.00,5.00,1.0,0.0,0.0,0.0,0.0,0.0,0.0
10851,2,5.00,5.00,5.00,5.00,5.00,5.00,5.00,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [44]:
# Create a StandardScaler instances.
scaler = StandardScaler()

# Fit the StandardScaler.
X_scaled = scaler.fit(X_train)

# Scale the data.
X_train_scaled = X_scaled.transform(X_train)
X_test_scaled = X_scaled.transform(X_test)

In [45]:
# Define the number of input features.
num_features = len(X.columns)

# Create the model
model_2 = tf.keras.models.Sequential()

# First hidden layer.
model_2.add(tf.keras.layers.Dense(units=12, input_dim=num_features, activation='relu'))

# Second hidden layer.
model_2.add(tf.keras.layers.Dense(units=16, activation='relu'))

# Output layer.
model_2.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model.
model_2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 12)                192       
                                                                 
 dense_4 (Dense)             (None, 16)                208       
                                                                 
 dense_5 (Dense)             (None, 1)                 17        
                                                                 
Total params: 417
Trainable params: 417
Non-trainable params: 0
_________________________________________________________________


In [46]:
# Compile the model.
model_2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [47]:
# Train the model.
fit_model_2 = model_2.fit(X_train, y_train, epochs=100)

Epoch 1/100
255/255 [==============================] - 1s 1ms/step - loss: 0.7316 - accuracy: 0.5089
Epoch 2/100
255/255 [==============================] - 0s 1ms/step - loss: 0.6887 - accuracy: 0.5406
Epoch 3/100
255/255 [==============================] - 0s 1ms/step - loss: 0.6862 - accuracy: 0.5502
Epoch 4/100
255/255 [==============================] - 0s 1ms/step - loss: 0.6862 - accuracy: 0.5520
Epoch 5/100
255/255 [==============================] - 0s 1ms/step - loss: 0.6852 - accuracy: 0.5552
Epoch 6/100
255/255 [==============================] - 0s 1ms/step - loss: 0.6845 - accuracy: 0.5542
Epoch 7/100
255/255 [==============================] - 0s 1ms/step - loss: 0.6851 - accuracy: 0.5545
Epoch 8/100
255/255 [==============================] - 0s 1ms/step - loss: 0.6837 - accuracy: 0.5574
Epoch 9/100
255/255 [==============================] - 0s 1ms/step - loss: 0.6835 - accuracy: 0.5617
Epoch 10/100
255/255 [==============================] - 0s 1ms/step - loss: 0.6836 - accura

In [48]:
# Evaluate the model using the test data.
model_2_loss, model_2_acc = model_2.evaluate(X_test, y_test)
print(f'Loss: {model_2_loss}, Accuracy: {model_2_acc}')

85/85 [==============================] - 0s 1ms/step - loss: 0.6773 - accuracy: 0.5660
Loss: 0.6773110628128052, Accuracy: 0.565954327583313


# Logistic Regression Model

In [49]:
classifier = LogisticRegression(solver='lbfgs', max_iter=200, random_state=1)
classifier

LogisticRegression(max_iter=200, random_state=1)

In [50]:
classifier.fit(X_train, y_train)

C:\Users\radov\anaconda3\envs\tfenv\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=200, random_state=1)

In [ ]:
print(f"Training Data Score: {classifier.score(X_train, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test, y_test)}")

In [ ]:
predictions = classifier.predict(X_test)
results = pd.DataFrame({"Prediction": predictions, "Actual": y_test}).reset_index(drop=True)
results

In [ ]:
accuracy_score(y_test, predictions)